In [1]:
# -*- coding: utf-8 -*-
"""
This example demonstrates the use of ImageView, which is a high-level widget for 
displaying and analyzing 2D and 3D data. ImageView provides:

  1. A zoomable region (ViewBox) for displaying the image
  2. A combination histogram and gradient editor (HistogramLUTItem) for
     controlling the visual appearance of the image
  3. A timeline for selecting the currently displayed frame (for 3D data only).
  4. Tools for very basic analysis of image data (see ROI and Norm buttons)

"""
## Add path to library (just for examples; you do not need this)
#import initExample

import numpy as np
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph as pg

# Interpret image data as row-major instead of col-major
pg.setConfigOptions(imageAxisOrder='row-major')

app = QtGui.QApplication([])

## Create window with ImageView widget
win = QtGui.QMainWindow()
win.resize(800,800)

imv = pg.ImageView(view=pg.PlotItem())
win.setCentralWidget(imv)
win.show()
win.setWindowTitle('pyqtgraph example: ImageView')

## Create random 3D data set with noisy signals
img = pg.gaussianFilter(np.random.normal(size=(200, 200)), (5, 5)) * 20 + 100
img = img[np.newaxis,:,:]
decay = np.exp(-np.linspace(0,0.3,100))[:,np.newaxis,np.newaxis]
data = np.random.normal(size=(100, 200, 200))
data += img * decay
data += 2

## Add time-varying signal
sig = np.zeros(data.shape[0])
sig[30:] += np.exp(-np.linspace(1,10, 70))
sig[40:] += np.exp(-np.linspace(1,10, 60))
sig[70:] += np.exp(-np.linspace(1,10, 30))

sig = sig[:,np.newaxis,np.newaxis] * 3
data[:,50:60,30:40] += sig

vLine = pg.InfiniteLine(angle=90, movable=False)
hLine = pg.InfiniteLine(angle=0, movable=False)
imv.addItem(vLine, ignoreBounds=True)
imv.addItem(hLine, ignoreBounds=True)


## Display the data and assign each frame a time value from 1.0 to 3.0
imv.setImage(data, xvals=np.linspace(1., 3., data.shape[0]))

## Set a custom color map
colors = [
    (0, 0, 0),
    (45, 5, 61),
    (84, 42, 55),
    (150, 87, 60),
    (208, 171, 141),
    (255, 255, 255)
]
cmap = pg.ColorMap(pos=np.linspace(0.0, 1.0, 6), color=colors)
imv.setColorMap(cmap)

def mouseMoved(evt):
    print(evt)
#     global data
#     global imv
#     pos = evt[0]  ## using signal proxy turns original arguments into a tuple
#     if imv.view.sceneBoundingRect().contains(pos):
#         mousePoint = imv.view.getViewBox().mapSceneToView(pos)
#         index = int(mousePoint.x())
#         if index > 0 and index < len(data):
#             print("{} and {}".format((mousePoint.x(), data[index])))
#         vLine.setPos(mousePoint.x())
#         hLine.setPos(mousePoint.y())


proxy = pg.SignalProxy(imv.view.scene().sigMouseMoved, rateLimit=60, slot=mouseMoved)




## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()


(PyQt5.QtCore.QPointF(614.0, 549.0),)
(PyQt5.QtCore.QPointF(538.0, 500.0),)
(PyQt5.QtCore.QPointF(477.0, 462.0),)
(PyQt5.QtCore.QPointF(434.0, 432.0),)
(PyQt5.QtCore.QPointF(410.0, 411.0),)
(PyQt5.QtCore.QPointF(394.0, 392.0),)
(PyQt5.QtCore.QPointF(386.0, 379.0),)
(PyQt5.QtCore.QPointF(382.0, 375.0),)
(PyQt5.QtCore.QPointF(382.0, 373.0),)
(PyQt5.QtCore.QPointF(385.0, 373.0),)
(PyQt5.QtCore.QPointF(400.0, 373.0),)
(PyQt5.QtCore.QPointF(438.0, 373.0),)
(PyQt5.QtCore.QPointF(481.0, 373.0),)
(PyQt5.QtCore.QPointF(516.0, 373.0),)
(PyQt5.QtCore.QPointF(522.0, 373.0),)
(PyQt5.QtCore.QPointF(523.0, 373.0),)
(PyQt5.QtCore.QPointF(522.0, 369.0),)
(PyQt5.QtCore.QPointF(516.0, 347.0),)
(PyQt5.QtCore.QPointF(507.0, 309.0),)
(PyQt5.QtCore.QPointF(505.0, 284.0),)
(PyQt5.QtCore.QPointF(505.0, 275.0),)
(PyQt5.QtCore.QPointF(505.0, 274.0),)
(PyQt5.QtCore.QPointF(504.0, 274.0),)
(PyQt5.QtCore.QPointF(489.0, 274.0),)
(PyQt5.QtCore.QPointF(446.0, 272.0),)
(PyQt5.QtCore.QPointF(397.0, 269.0),)
(PyQt5.QtCor